TED talk text generator

Generate using a character based model.

The code is this notebook is a partial modification of several tutorials, blog posts and YouTube videos. The main sources include:
* [Text Generation With LSTM Recurrent Neural Networks in Python with Keras](https://machinelearningmastery.com/text-generation-lstm-recurrent-neural-networks-python-keras/) by Dr. Jason Brownlee
* [Text Generation with Python and TensorFlow/Keras](https://stackabuse.com/text-generation-with-python-and-tensorflow-keras/) by Dan Nelson
* YouTube: [167 - Text prediction using LSTM (English text)](https://youtu.be/zyCpntcVKSo?t=2) by Sreenivas B.


In [161]:
import numpy as np
from tensorflow.keras.utils import to_categorical
from tensorflow.train import latest_checkpoint
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Dropout, LSTM
import random
import textwrap

In [162]:
# Attach to my Google drive so I can save the csv file later
# from google.colab import drive
# drive.mount('/content/drive')

In [163]:
#tag_name = 'all_transcripts'
#tag_name = 'brain'
#tag_name = 'business'
#tag_name = 'creativity'
tag_name = 'culture'
#tag_name = 'psychology'
#tag_name = 'science'

lower_corpus = False

if lower_corpus:
    corpus_file_name = tag_name + '_lowercase'
else:
    corpus_file_name = tag_name + '_sentence_case'

# This switch makes it easier running locally or in a Google Colab environment
colab = False

if colab:
    root_path = '/content/drive/MyDrive/ted/'
else:
    root_path = '../data/'
    
sequence_length = 80

In [164]:
with open(f'{root_path}{corpus_file_name}.txt', encoding='utf-8') as f:
    corpus = f.read()
f.close()

In [165]:
corpus[0:500]

"Good morning. How are you?  Good. It's been great, hasn't it? I've been blown away by the whole thing. In fact, I'm leaving.  There have been three themes running through the conference, which are relevant to what I want to talk about. One is the extraordinary evidence of human creativity in all of the presentations that we've had and in all of the people here; just the variety of it and the range of it. The second is that it's put us in a place where we have no idea what's going to happen in te"

The code below is used in almost every article I have seen about LSTM for text generation, but this snippet is adapted from :
https://www.kaggle.com/mrisdal/intro-to-lstms-w-keras-gpu-for-text-generation/

In [166]:
all_chars = sorted(list(set(corpus)))

#remove most punctuation, but leave spaces, exclamation points, and periods
list_to_remove = ['"', '&', '-', ',', "'", '/', ';',  '—']

chars = [character for character in all_chars if character not in list_to_remove]
print(chars)
# hat tip to this post for the idea of removing one list from another
# https://www.geeksforgeeks.org/python-remove-all-values-from-a-list-present-in-other-list/

[' ', '!', '%', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 'é']


In [167]:
# remove all unwanted characters from the original corpus
corpus_list = [character for character in corpus if character not in list_to_remove]

# Re-join all letters back into a single string as the updated corpus
corpus = ''.join(corpus_list)

In [168]:
# If running models with punctuation removed, leave commented out
#chars = sorted(list(set(corpus)))

print('Number of unique characters in the corpus:', len(chars))

char_indices = dict((c, i) for i, c in enumerate(chars))

indices_char = dict((i, c) for i, c in enumerate(chars))

Number of unique characters in the corpus: 67


In [169]:
# Show character to index mapping
print (char_indices)

{' ': 0, '!': 1, '%': 2, '.': 3, '0': 4, '1': 5, '2': 6, '3': 7, '4': 8, '5': 9, '6': 10, '7': 11, '8': 12, '9': 13, '?': 14, 'A': 15, 'B': 16, 'C': 17, 'D': 18, 'E': 19, 'F': 20, 'G': 21, 'H': 22, 'I': 23, 'J': 24, 'K': 25, 'L': 26, 'M': 27, 'N': 28, 'O': 29, 'P': 30, 'R': 31, 'S': 32, 'T': 33, 'U': 34, 'V': 35, 'W': 36, 'X': 37, 'Y': 38, 'Z': 39, 'a': 40, 'b': 41, 'c': 42, 'd': 43, 'e': 44, 'f': 45, 'g': 46, 'h': 47, 'i': 48, 'j': 49, 'k': 50, 'l': 51, 'm': 52, 'n': 53, 'o': 54, 'p': 55, 'q': 56, 'r': 57, 's': 58, 't': 59, 'u': 60, 'v': 61, 'w': 62, 'x': 63, 'y': 64, 'z': 65, 'é': 66}


In [170]:
# Show index to character mapping
print(indices_char)

{0: ' ', 1: '!', 2: '%', 3: '.', 4: '0', 5: '1', 6: '2', 7: '3', 8: '4', 9: '5', 10: '6', 11: '7', 12: '8', 13: '9', 14: '?', 15: 'A', 16: 'B', 17: 'C', 18: 'D', 19: 'E', 20: 'F', 21: 'G', 22: 'H', 23: 'I', 24: 'J', 25: 'K', 26: 'L', 27: 'M', 28: 'N', 29: 'O', 30: 'P', 31: 'R', 32: 'S', 33: 'T', 34: 'U', 35: 'V', 36: 'W', 37: 'X', 38: 'Y', 39: 'Z', 40: 'a', 41: 'b', 42: 'c', 43: 'd', 44: 'e', 45: 'f', 46: 'g', 47: 'h', 48: 'i', 49: 'j', 50: 'k', 51: 'l', 52: 'm', 53: 'n', 54: 'o', 55: 'p', 56: 'q', 57: 'r', 58: 's', 59: 't', 60: 'u', 61: 'v', 62: 'w', 63: 'x', 64: 'y', 65: 'z', 66: 'é'}


In [171]:
print (f'The input corpus contains {len(corpus):,} characters.')

The input corpus contains 118,864 characters.


The model will use arbitrary length of characters, e.g. 40, and then predict the next character that will appear (the 41st).

I would like to change this sequence length in different models, to see how it might affect the model's performance.

Larger might be better?

Loop through the corpus, creating segments of 40 characters, plus a segment of a single character that would appear after it. These segments are then converted into their respective digits, and loaded into X

In addition, create a list of target values. The target, y, is a single text character, and is also converted into a corresponding numeric value.

In [172]:
corpus[0:50]

'Good morning. How are you?  Good. Its been great h'

In [173]:
corpus[1:41]

'ood morning. How are you?  Good. Its bee'

In [174]:
corpus[41]

'n'

In [175]:
# For future modeling, we could try to offset the sequences by more than one
# letter. For now the model will look at every sequence right after another.
step = 1

X_numeric_list = []
y_numeric_list = []


for i in range (0, len(corpus) - sequence_length, step):
    # To find X when still as characters, loop through and extract
    # a sequence for example, from the 2nd to the 42nd, as in corpus[1:41] since 
    # the string index is zero-based
    X_char_sequence = corpus[i:i + sequence_length]  #exclusive
    y_char = corpus[i + sequence_length]

    # Convert the X character sequence into a list of integers, using the 
    # dictionary created above.
    X_numeric_list.append( [char_indices[letter] for letter in X_char_sequence])
    # Also convert target letter y to it's corresponding numeric value in the
    # dictionary
    y_numeric_list.append(char_indices[y_char])



In [176]:
# Look at the first two converted sequences
print ( X_numeric_list[0])
print ( X_numeric_list[1])


[21, 54, 54, 43, 0, 52, 54, 57, 53, 48, 53, 46, 3, 0, 22, 54, 62, 0, 40, 57, 44, 0, 64, 54, 60, 14, 0, 0, 21, 54, 54, 43, 3, 0, 23, 59, 58, 0, 41, 44, 44, 53, 0, 46, 57, 44, 40, 59, 0, 47, 40, 58, 53, 59, 0, 48, 59, 14, 0, 23, 61, 44, 0, 41, 44, 44, 53, 0, 41, 51, 54, 62, 53, 0, 40, 62, 40, 64, 0, 41]
[54, 54, 43, 0, 52, 54, 57, 53, 48, 53, 46, 3, 0, 22, 54, 62, 0, 40, 57, 44, 0, 64, 54, 60, 14, 0, 0, 21, 54, 54, 43, 3, 0, 23, 59, 58, 0, 41, 44, 44, 53, 0, 46, 57, 44, 40, 59, 0, 47, 40, 58, 53, 59, 0, 48, 59, 14, 0, 23, 61, 44, 0, 41, 44, 44, 53, 0, 41, 51, 54, 62, 53, 0, 40, 62, 40, 64, 0, 41, 64]


In [177]:
print ( y_numeric_list[0:2])

[64, 0]


In [178]:
indices_char[51]

'l'

In [179]:
corpus[0:43]

'Good morning. How are you?  Good. Its been '

This example is a little hard to follow, since we happened to be in the middle of the word 'been'. The letter e is mapped to 51.

However the beginning of the sequence has been shifted by one as well.

In [180]:
print (f'There are {len(X_numeric_list):,} entries in X_numeric_list') 

There are 118,784 entries in X_numeric_list


An LSTM model needs the X data to be in 3 dimensions:
* Samples (number of rows)
* Time steps (this implies time series. In this example this corresponds to the sequence_length)
* Features (the target, or y variable)

This [blog post by Dr. John Brownlee](https://machinelearningmastery.com/reshape-input-data-long-short-term-memory-networks-keras/) contains a longer explanation, which I paraphrased here.

In [181]:
# reshape X to be [samples, time steps, features]
# Samples is the total number of 40 character text fragments created above
# Time step is the sequence length, which is this case is 40, but might change
# in other model variations
# Feature is one, since the model is predicting one character at a time
X = np.reshape(X_numeric_list, (len(X_numeric_list), sequence_length, 1))

In [182]:
X.shape

(118784, 80, 1)

In [183]:
# normalize/scale the data by dividing by the length of the character list.
X = X / len(chars)

In [184]:
# Look at the first value of X
X[0]

array([[0.31343284],
       [0.80597015],
       [0.80597015],
       [0.64179104],
       [0.        ],
       [0.7761194 ],
       [0.80597015],
       [0.85074627],
       [0.79104478],
       [0.71641791],
       [0.79104478],
       [0.68656716],
       [0.04477612],
       [0.        ],
       [0.32835821],
       [0.80597015],
       [0.92537313],
       [0.        ],
       [0.59701493],
       [0.85074627],
       [0.65671642],
       [0.        ],
       [0.95522388],
       [0.80597015],
       [0.89552239],
       [0.20895522],
       [0.        ],
       [0.        ],
       [0.31343284],
       [0.80597015],
       [0.80597015],
       [0.64179104],
       [0.04477612],
       [0.        ],
       [0.34328358],
       [0.88059701],
       [0.86567164],
       [0.        ],
       [0.6119403 ],
       [0.65671642],
       [0.65671642],
       [0.79104478],
       [0.        ],
       [0.68656716],
       [0.85074627],
       [0.65671642],
       [0.59701493],
       [0.880

Even thought the values for the target variable are integers, they are essentially labels for the predicted characters. They are one-hot encoded using the to_categorical function.

In [185]:
y = to_categorical(y_numeric_list)
y[0]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
      dtype=float32)

---
### Load LSTM Model
In this notebook, load a model that was previously trained and saved.

In [186]:
# Save the model to my Google Drive so I can load it later from another notebook
# model = load_model(f'/content/drive/MyDrive/ted/models/ted_model_{tag_name}')

# model.summary()

If a model failed during fitting, we can still load it's most recent weights which were stored in a checkpoint file.

Use of ModelCheckPoint to save the model's progress example as described in the [TensorFlow documentation.](https://www.tensorflow.org/tutorials/keras/save_and_load#checkpoint_callback_options)

In [187]:
checkpoint_dir = f'{root_path}{corpus_file_name}/'

latest = latest_checkpoint(checkpoint_dir)


In [188]:
# On Colab, model saved as 
# model.save(f'/content/drive/MyDrive/ted/models/ted_model_{tag_name}_seq_{sequence_length}')

In [189]:
print ( f'{root_path}models/ted_model_{tag_name}')

../data/models/ted_model_culture


In [190]:
# define the LSTM model
# model = Sequential()
# model.add(LSTM(1000, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
# model.add(Dropout(0.2))
# model.add(LSTM(256))
# model.add(Dropout(0.2))
# model.add(Dense(y.shape[1], activation='softmax'))


# # Load the previously saved weights
# model.load_weights(latest)

# model.compile(loss='categorical_crossentropy', optimizer='adam' , metrics = ['accuracy','Recall','Precision'])
#model = load_model(f'{latest}')

# Use if there was a successful save of a model

model = load_model(f'{root_path}models/ted_model_{tag_name}')

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 80, 800)           2566400   
_________________________________________________________________
dropout (Dropout)            (None, 80, 800)           0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 512)               2689024   
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense (Dense)                (None, 67)                34371     
Total params: 5,289,795
Trainable params: 5,289,795
Non-trainable params: 0
_________________________________________________________________


In [191]:
model.count_params

<bound method Layer.count_params of <tensorflow.python.keras.engine.sequential.Sequential object at 0x000002B8D9B44FA0>>

X_numeric_list contains all of our 40 character sequences. Choose one for a seed for the text generation.

In [192]:
seed_start = random.randint(0, len(corpus) - sequence_length)
print(f'Random start index for seed text: {seed_start} sequence.')

X_seeded = X_numeric_list[seed_start]
print(X_seeded)

Random start index for seed text: 47296 sequence.
[0, 0, 0, 47, 40, 58, 0, 40, 0, 51, 48, 53, 44, 0, 48, 53, 0, 47, 48, 58, 0, 41, 54, 54, 50, 3, 0, 23, 0, 51, 54, 61, 44, 43, 0, 59, 47, 48, 58, 0, 46, 60, 64, 3, 0, 23, 0, 46, 54, 59, 0, 40, 0, 51, 54, 59, 0, 54, 45, 0, 52, 48, 51, 44, 40, 46, 44, 0, 54, 60, 59, 0, 54, 45, 0, 33, 47, 44, 54, 43]


Convert this back to text so we can see what it looks like.

In [193]:
seed_string = ''.join([indices_char[letter_code] for letter_code in X_seeded])
seed_string

'   has a line in his book. I loved this guy. I got a lot of mileage out of Theod'

In [194]:
# # We can also use the seed_start value to see a larger picture of the original text
# for j in range(seed_start, seed_start + 15):
#     seed_numeric = X_numeric_list[j]
#     print (''.join([indices_char[letter_code] for letter_code in seed_numeric]))

To generate text, we need to change the input text into a 3D shape as we did the X input data.

In [195]:
temp_x = np.reshape(X_seeded, (1, len(X_seeded) , 1))
# normalize/scale the data by dividing by the length of the character list.
temp_x = temp_x / len(chars)
prediction = model.predict(temp_x, verbose=0)
print(f"A prediction's shape:{prediction.shape}")
print('Prediction:')
print (prediction)

A prediction's shape:(1, 67)
Prediction:
[[4.3914006e-03 1.1104835e-06 2.1210081e-11 8.2501705e-05 2.4417057e-09
  6.8372759e-11 6.7444561e-12 3.1936870e-11 2.7406258e-10 1.7842514e-09
  2.8537258e-11 2.2655761e-10 1.5724758e-11 2.0451969e-10 1.5616301e-06
  1.8109164e-08 6.4257442e-08 4.6327767e-08 2.7143483e-07 1.4027248e-07
  4.4227724e-10 4.0851603e-08 5.5459104e-09 1.0591317e-06 5.8205891e-11
  1.5441918e-08 1.5058824e-08 2.9912596e-06 2.5817421e-08 7.4611872e-09
  6.9332549e-09 3.7343759e-07 3.9866691e-07 1.3099294e-07 1.1953251e-08
  5.9499898e-11 3.6672784e-10 5.7079765e-11 7.9571045e-08 1.2868332e-09
  6.1459315e-04 4.7457554e-05 2.5962825e-06 3.2348751e-06 5.8869983e-04
  2.3734073e-04 4.6220248e-05 6.2645522e-05 7.6273594e-05 2.1324895e-04
  1.2504979e-05 1.3424276e-04 3.1721465e-06 6.1864002e-06 9.8834330e-01
  1.6968287e-04 3.2466470e-08 1.0403246e-03 5.1418535e-04 1.5865023e-04
  3.6265215e-04 6.6148183e-05 2.8114642e-03 2.8770009e-08 2.6970149e-06
  2.0497575e-07 2.17440

Each prediction produces an result of shape (1, 75), and contains a predicted probability of a character. Use argmax to find the index of the highest probability.

In [196]:
np.argmax(prediction)

54

So the first letter predicted after the seed sequence is:

In [197]:
indices_char[np.argmax(prediction)]

'o'

In [198]:
# Put all predicted characters in a list, then convert to a string later.
complete_predicted_text =[]

# Generate 1000 characters
for i in range(600):
    # reshape to 1 row, sequence length, and 1 for predicting one character as the target
    temp_x = np.reshape(X_seeded, (1, len(X_seeded) , 1))
    # normalize/scale the data by dividing by the length of the character list.
    temp_x = temp_x / len(chars)
    prediction = model.predict(temp_x, verbose=0)
    complete_predicted_text.append(indices_char[np.argmax(prediction)])

    # After predicting, add this character's numeric value on to the randomly
    # chose seed string, and use this for another prediction
    X_seeded.append(np.argmax(prediction))

    # Move the seeded text up one character, so create a new sequence
    X_seeded = X_seeded[1:len(X_seeded)]


#print (complete_predicted_text)


The predicted results are a list of characters, so convert into a string

In [199]:
predicted_string = ''.join(complete_predicted_text)

# Wrap to 120 characters just so I can visually compare to the original string.
textwrap.wrap(seed_string + '->'+ predicted_string, width=120)

['   has a line in his book. I loved this guy. I got a lot of mileage out of Theod->oor van De Velde. He had this line in',
 'his book that suppers to think  In not mandsts with all she willens ot whth the artinacee th speak but to salk about the',
 'world in ooe our of the plomes that we are doing tiat in a lear  I tritt to this because then you work teelny imsriver',
 'fer eutencty. In fact I did it first in a food in  The hostanee os thich were being somernde wou know that the torld we',
 'thenk you fird this  And she was a psufe same I was going to sea tome out of the fight. That if my roommate knew about',
 'the weather hiw rex withing that we are doing. And thats the prrclems that we goow']

In [200]:
# Find the original seed string to see what came after it
original_paragraph = corpus[corpus.find(seed_string): corpus.find(seed_string) + 2000]

textwrap.wrap(original_paragraph, width=120)

['   has a line in his book. I loved this guy. I got a lot of mileage out of Theodoor van De Velde. He had this line in',
 'his book that supposedly comes from the Habsburg Monarchy where there was an empress Maria Theresa who was having',
 'trouble conceiving. And apparently the royal court physician said to her I am of the opinion that the vulva of your most',
 'sacred majesty be titillated for some time prior to intercourse.  Its apparently I dont know on the record somewhere.',
 'Masters and  now were moving forward to the 1950s. Masters and Johnson were upsuck skeptics which is also really fun to',
 'say. They didnt buy it. And they decided being Masters and Johnson that they would get to the bottom of it. They brought',
 'women into the lab  I think it was five women  and outfitted them with cervical caps containing artificial semen. And in',
 'the artificial semen was a radioopaque substance such that it would show up on an Xray. This is the 1950s. Anyway these',
 'women sat in 